<a href="https://colab.research.google.com/github/Catisyf/Python-projects-chez-Toucan/blob/main/Backfill_text_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls '/content/drive/My Drive/crm_contacts.csv'

'/content/drive/My Drive/crm_contacts.csv'


In [ ]:
pip install -U sentence-transformers

In [ ]:
import pandas as pd
from scipy import spatial
from collections import Counter

In [214]:
df_contacts = pd.read_csv('/content/drive/My Drive/crm_contacts.csv', na_values=' ') #replace blank cells with na
df_persona = pd.read_csv('/content/drive/My Drive/buyer_persona_definition.csv')

In [220]:
df_persona

,buyer_persona,job_title
0,"Data Leader - CIO, CDO",CIO
1,"Data Leader - CIO, CDO",CDO
2,"Data Leader - CIO, CDO",Chief Information Officer
3,"Data Leader - CIO, CDO",CIO
4,"Data Leader - CIO, CDO",Head of IT
...,...,...
126,Practitioner,Chef de Projet Senior BI
127,Practitioner,Chef de Projet Senior Business Intelligence
128,Practitioner,Chef de Projet Senior Insights
129,Practitioner,Chef de Projet Senior Analytics


## **Data Preparation**

Step 1: remove null values & values that contain '(site)' from CRM data

Step 2: remove stop words in both datasets (of, de). It is not necessary to use stopwords dictionaries.  

In [215]:
df_contacts = df_contacts.replace('?', 'np.Nan').dropna() #remove null

In [216]:
df_contacts = df_contacts[~df_contacts.jobtitle.str.contains('(site)')] #remove strings that contain (site)

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [218]:
stopwords = ['of', 'de', 'du', 'for', 'For', 'the', 'et', '&', '/'] #may consider using the EN & FR stopwords
space = ' '
stopwords_dict = Counter(stopwords)
for n in range(0,len(df_contacts)):
  df_contacts.iloc[n]['jobtitle'] = space.join([word for word in df_contacts.iloc[n]['jobtitle'].split() \
                                                if word not in stopwords_dict]) #remove stopwords for crm data

In [221]:
for n in range(0,len(df_persona)):
  df_persona.iloc[n]['job_title'] = space.join([word for word in df_persona.iloc[n]['job_title'].split() \
                                                if word not in stopwords_dict]) #remove stopwords for persona

In [ ]:
#Pre-trained sentence transformer model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [266]:
#MOCK dataset
#doc1 = ['BI Analyst', 'Data Analyst', 'head product', 'director marketing']
#doc2 = ['BI engineer', 'Business Analyst', 'marketing manager', 'chef produit']

In [233]:
#Sentences are encoded by calling model.encode()
vector1 = []
for i in range(0,len(df_contacts)):
  vector1.append(model.encode(df_contacts.iloc[i]['jobtitle']))

df_contacts['vector'] = vector1

In [231]:
vector2 = []
for i in range(0,len(df_persona)):
  vector2.append(model.encode(df_persona.iloc[i]['job_title']))

df_persona['vector'] = vector2

In [ ]:
#def cosine_similarity (x,y):

In [ ]:
for vec1 in df_contacts['vector']:
  for vec2 in df_persona['vector']:
    if (1-spatial.distance.cosine(vec1, vec2)) >= 0.7:
      print(vec1, vec2, 1-spatial.distance.cosine(vec1, vec2))